In [1]:
import os
import scipy.io
import numpy as np
import pandas as pd
import pickle

In [10]:
def mat_struct_to_dict(mat_struct):
    """
    Converte uma struct MATLAB (com subcampos) em um dicionário de dicionários.
    """
    if not isinstance(mat_struct, np.ndarray) or mat_struct.dtype.names is None:
        return mat_struct  # Retorna o valor direto se não for uma struct

    resultado = {}
    for nome_campo in mat_struct.dtype.names:
        valor_campo = mat_struct[nome_campo][0, 0]  # Assume struct 1x1
        # Chamada recursiva para processar subcampos
        resultado[nome_campo] = mat_struct_to_dict(valor_campo)  
    return resultado

In [8]:
def data_formatter(original_path, save_path, matlab_var_name):
    eeg = scipy.io.loadmat(original_path)
    new_dict = mat_struct_to_dict(eeg[matlab_var_name])
    new_df = pd.DataFrame.from_dict(new_dict, orient='columns')
    
    with open(save_path, 'wb') as arquivo:
            pickle.dump(new_df, arquivo)

In [36]:
#formata os dados sem csd
for subject_number in range(1,16):
    # subject_number = 1
    current_dir = os.path.dirname(os.path.abspath('.'))
    path = os.path.join(current_dir, 'EEG-SUBAMOSTRADO', 'EEG dividido em períodos', 'periodos_EEG_Subject'+str(subject_number)+'.mat')
    nome_arquivo = 'Subject'+str(subject_number)+'.pkl'
    save_path = os.path.join(current_dir,'ML','Original', nome_arquivo)
    data_formatter(path,save_path,'periodos')

In [38]:
# formata os dados com csd
for subject_number in range(1,16):
    # subject_number = 1
    current_dir = os.path.dirname(os.path.abspath('.'))
    path = os.path.join(current_dir, 'EEG-SUBAMOSTRADO', 'EEG com CSD', 'Subject'+str(subject_number)+'_CSD.mat')
    nome_arquivo = 'Subject'+str(subject_number)+'_csd.pkl'
    save_path = os.path.join(current_dir,'ML','CSD', nome_arquivo)
    data_formatter(path,save_path,'eeg_csd')

In [3]:
# aqui há a formatação a partir dos arquivos com a banda já extraída
for banda in ['alfa', 'beta','gama', 'teta']:
    for subject_number in ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15']:
        # subject_number = '1'
        current_dir = os.path.dirname(os.path.abspath('.'))
        original_path = os.path.join(current_dir, 'EEG-SUBAMOSTRADO','Bandas', 'Original', banda, 'Subject'+subject_number+'nocsd_'+banda+'.mat')
        csd_path = os.path.join(current_dir, 'EEG-SUBAMOSTRADO','Bandas', 'CSD', banda, 'Subject'+subject_number+'_'+banda+'.mat')

        eeg_original = scipy.io.loadmat(original_path)
        eeg_csd = scipy.io.loadmat(csd_path)
        new_dict_original = mat_struct_to_dict(eeg_original['eeg_'+ banda])
        new_dict_csd = mat_struct_to_dict(eeg_csd['eeg_'+ banda])
    
        new_df_original = pd.DataFrame.from_dict(new_dict_original, orient='columns')
        new_df_csd = pd.DataFrame.from_dict(new_dict_csd, orient='columns')

        nome_arquivo_original = 'Subject'+subject_number+'_'+banda+'.pkl'
        nome_arquivo_csd = 'Subject'+subject_number+'_CSD'+'_'+banda+'.pkl'

        save_orginal_path = os.path.join(current_dir,'ML','Original', 'Bandas', banda, nome_arquivo_original)
        save_csd_path = os.path.join(current_dir,'ML','CSD', 'Bandas',banda, nome_arquivo_csd)

        with open(save_orginal_path, 'wb') as arquivo:
            pickle.dump(new_df_original, arquivo)

        with open(save_csd_path, 'wb') as arquivo:
            pickle.dump(new_df_csd, arquivo)

In [17]:
df_test = pd.DataFrame.from_dict(new_dict_original, orient='columns')
df_test

,bloco_ternura1,bloco_angustia1,bloco_ternura2,bloco_angustia2,bloco_ternura3,bloco_angustia3,bloco_ternura4,bloco_angustia4
neutro1,"[[1.0516211974767045, 0.8726242088177409, 0.49...","[[-0.041833066027207995, 0.4383640088591614, -...","[[0.2530028760063298, 0.15374877619543928, 0.0...","[[-5.122995020051867, -5.049450958691308, -0.3...","[[-0.865112506490269, -1.111753353961235, -0.1...","[[-0.7961261510618438, -1.1077753369375833, 0....","[[1.5431390465982486, 0.68153818237079, -0.140...","[[1.8995948229404411, 2.6571911860398747, -0.5..."
ternura1,"[[0.058994205027678465, 0.2386602432910141, -0...",NaN,"[[-0.02857053939899412, 0.1478766424768923, 0....",NaN,"[[-0.13314511799851006, -0.43240470534067227, ...",NaN,"[[-0.7036966408902992, -1.569756459771765, -0....",NaN
neutro2,"[[-0.321875332940103, 0.10633892341149198, -0....","[[-2.725382585246722, -2.1818208851689196, -0....","[[13.430393829399282, 13.760626758098738, 2.73...","[[-0.47386029961085596, -0.6294932504279911, -...","[[-0.16192878182800963, -0.5188995605497361, -...","[[-0.23878938804190628, -0.42548257159725716, ...","[[0.3369829145781946, 0.7652865210025124, -0.0...","[[-0.5584215602437063, 0.03461885679877423, -0..."
ternura2,"[[-0.2346124579510797, -0.12157099117255628, -...",NaN,"[[0.18512210211299987, 0.013501927701787745, 0...",NaN,"[[-1.2838936261493938, -0.2815141457295301, -0...",NaN,"[[0.9833957116488344, 0.33145466494937936, 1.0...",NaN
neutro3,"[[-4.2402395587359205, -4.587307367478843, -0....","[[0.012663108033286633, -0.8218897894725271, -...","[[-3.1510591157065715, -2.9107513027785314, -1...","[[0.5468743507715347, 1.0707186856344262, 0.66...","[[-1.157103134630487, -0.6107539508114592, -0....","[[-0.39846666024577215, -0.036568529223771955,...","[[-0.2869016299075546, -0.11905546354183062, 0...","[[0.07387872519776013, -0.2812178855647436, 0...."
ternura3,"[[-0.1542264899605405, -0.08371830260744799, 0...",NaN,"[[-0.36357896955533187, -0.015279465715221294,...",NaN,"[[-0.01689099674485943, 0.16526352834431154, 0...",NaN,"[[0.6409989676715957, -0.035617837477786525, 0...",NaN
neutro4,"[[0.5114122942048168, 0.43813712165171026, 0.5...","[[9.225340925243126, 9.240350120735684, 1.2793...","[[-0.3277279196869567, 0.06584600372423474, 0....","[[-0.17477191858191285, -0.16103526714535638, ...","[[-0.3068041692207245, -0.35053724592982866, 0...","[[0.18105494475515443, -0.3543296711666104, 0....","[[-0.27262174110255954, 0.04489964022442534, 0...","[[0.9244975183295427, 0.05727708870699848, 0.5..."
ternura4,"[[-1.0370879380700149, -0.16464964751710495, -...",NaN,"[[-0.42076736853877594, 0.029964820700137337, ...",NaN,"[[-0.5464330858665551, -1.6219769714674726, -0...",NaN,"[[-0.1409601115322967, 0.05612068747816323, -0...",NaN
neutro5,"[[3.9358952355085974, 3.308947967614069, 0.893...","[[0.7478441721321412, 0.3966295283154515, 0.38...","[[3.5967777622767043, 4.204621625904503, 0.118...","[[-0.802392378533865, -1.2389612214313352, 0.0...","[[-0.1920775708638838, -0.356064605857282, 0.3...","[[0.645140244175014, -0.2033221906528327, 0.00...","[[-0.3363842502628774, -0.3926591558072376, -0...","[[0.21863523971768714, 0.45513653385511454, -0..."
angustia1,NaN,"[[-0.06266935961278354, 0.08446279062623882, 0...",NaN,"[[-0.22047263500250638, 0.616944149440921, -0....",NaN,"[[2.899748975955005, 2.8596347523397863, 0.972...",NaN,"[[0.7489060785785889, 0.5802967658715, 0.39252..."


In [85]:
# verificar matrizes de struct p/ dict
print(np.array_equal(loaded_dict['bloco_ternura1']['ternura3'],new_struct_csd['bloco_ternura1']['ternura3']))

True


In [86]:
# caso seja necessário transpor as matrizes posteriormente
def transpose_all_eeg(dict):
    new_dict = {}
    for bloco in dict.keys():
        new_dict[bloco]={}
        for trecho in dict[bloco].keys():
            new_dict[bloco][trecho] = dict[bloco][trecho].transpose()
    return new_dict

In [62]:
new_dict = transpose_all_eeg(new_struct) # tranformando os canais de colunas para linhas

In [65]:
# verificador se matrizes tranpostas estão corretas
print(np.array_equal(new_dict['bloco_ternura1']['ternura3'], new_struct['bloco_ternura1']['ternura3'].transpose()))

True
